In [25]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time

from pathlib import Path

from datetime import datetime
from sklearn.model_selection import train_test_split

import warnings # necessary b/c pandas & statsmodels datetime issue
warnings.simplefilter(action="ignore")

# Import Data
#### First we import a few different lists of Disney films.  We have lists from Wikipedia, D23 (The official Disney Fan Club), and Disney.com. The lists contain different titles and film data. We'll attempt to create a single master list of Disney films to work from.

### Import Disney.com data (All Films)

It looks like tge remakes may be indicated by the inclusion of the year of release in the title. <mark> Will need to determine if that is always the case and use the re-release as metadata</mark>  <br>

In [55]:
FL_Disney = pd.read_csv('DD_Films_List_Disney_Com/Film_list_Disney_com.csv')
FL_Disney.rename(     columns=({ 'Movie Title': 'title'}),     inplace=True )

print(FL_Disney.shape)
FL_Disney.head()

(657, 1)


,title
0,101 Dalmatians
1,101 Dalmatians (1996)
2,101 Dalmatians II: Patch's London Adventure
3,102 Dalmatians
4,"20,000 Leagues Under the Sea"


### Import D23 data (All Films)

This one column hold the film title, year of release, and Motion Picture Association film rating. <mark> Will need to seperate those before moving forward</mark>  <br>
Also, the D23 list (Disney's Official Fan Club) has almost 100 more titles that the Disney.com list.  657 to 748.  <mark> Will have to create a list of different titles and research further</mark>

In [56]:
FL_D23 = pd.read_csv('DD_Films_List_D23/D23_list.csv', header=None )
FL_D23.rename(     columns=({ '0': 'title'}),     inplace=True )
print(FL_D23.shape)
FL_D23.head()

(749, 1)


,0
0,1. 1937: Snow White and the Seven Dwarfs (G)
1,2. 1940: Pinocchio (G)
2,3. 1940: Fantasia (G)
3,4. 1941: The Reluctant Dragon
4,5. 1941: Dumbo (G)


### Import Wikipedia data (All Films)

Wikipedia's list includes a lot more information, but only 425 titels, as compared to 657 from Disney.com and 748 from D23.  <mark> Will have to create a list of different titles and research further</mark>

In [38]:
FL_WK = pd.read_csv('DD_Films_LIst_WK/FIlms_List_WK.csv', )
print(FL_WK.shape)
FL_WK.head()

(425, 10)


,US Release,Other Release Date,Title,Co-production companies,Category,Direct to video or streaming exclusive Disney+,Premium video on demand release through Disney+,Simultaneous release to theatres and on premium video on demand,non-US Film,Notes
0,5/19/1937,NaN,Academy Award Review of Walt Disney Cartoons,NaN,Animated feature,NaN,NaN,NaN,NaN,NaN
1,12/4/1939,NaN,Snow White and the Seven Dwarfs,NaN,Animated feature,NaN,NaN,NaN,NaN,NaN
2,6/1/1940,NaN,Pinocchio,NaN,Animated feature,NaN,NaN,NaN,NaN,NaN
3,10/23/1941,NaN,Dumbo,NaN,Animated feature,NaN,NaN,NaN,NaN,NaN
4,11/2/1941,NaN,Bambi,NaN,Animated feature,NaN,NaN,NaN,NaN,NaN


### Import Wikipedia data (Animated Films)

Wikipedia's list of Disney anitmated films.  A lot of movie info, but only 144 titles.  <mark> Will have to combine the movie data in to the master list</mark>

In [39]:
FL_WK_Animated = pd.read_csv('DD_Animated_List_WK/Disney_Animated_List_wk.csv', )
print(FL_WK_Animated.shape)
FL_WK_Animated.head()

(144, 14)


,Title,Original U.S. theatrical release date[rls 1],Other Theatrical Release Date,Animation studio[st 2],CoCredit,Released By,Film Type,Live-action / Animation hybrid,"Not produced by Disney, but released under its label.",US Release Exceptions,Released under the Touchstone Pictures label,Released by Disney outside North America,Released by Miramax Films when the studio was a subsidiary of Disney at the time of release,Release Note
0,Academy Award Review of Walt Disney Cartoons,5/19/1937,NaN,Walt Disney Animation Studios (1937 - Present),NaN,RKO Radio Pictures,Animation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,12/21/1937,NaN,Walt Disney Animation Studios (1937 - Present),NaN,RKO Radio Pictures,Animation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pinocchio,2/7/1940,NaN,Walt Disney Animation Studios (1937 - Present),NaN,RKO Radio Pictures,Animation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fantasia,11/13/1940,1/29/1941,Walt Disney Animation Studios (1937 - Present),NaN,Walt Disney Productions / RKO Radio Pictures,Animation,Live-action / Animation hybrid,NaN,NaN,NaN,NaN,NaN,Originally distributed by Walt Disney Producti...
4,The Reluctant Dragon,6/20/1941,NaN,Walt Disney Animation Studios (1937 - Present),NaN,RKO Radio Pictures,Animation,Live-action / Animation hybrid,NaN,NaN,NaN,NaN,NaN,NaN
